In [1]:
import pandas as pd
import numpy as np
import polars as pl
import os
import re
pd.set_option('display.max_columns',None)
import warnings
warnings.filterwarnings('ignore');

#### If We Have Multiple Master File Then First Append Them By Asking Path of Each

In [2]:
#### If We Have Multiple Master File Then First Append Them By Asking Path of Each
def append_files(n: int):
    collect_df = []
    for i in range(n):
        while True:
            _format = input('Enter the File Format (.excel/.csv): ').lower().strip()

            if not _format:
                print('Format Input Cannot Be Blank')
            elif _format not in ['.csv', '.excel']:
                print('Invalid File Format Provided, please choose (.excel/.csv)')
            else:
                file_path = input('Enter the Absolute Path Of the File: ').strip()

                if not file_path:
                    print("File Path Cannot Be Empty")
                else:
                    try:
                        if _format == '.csv':
                            df=pd.read_csv(file_path)
                        elif _format == '.excel':
                            df=pd.read_excel(file_path)

                        # Assuming `collect_df` is a list to store DataFrames
                        collect_df.append(df)
                        break
                    except Exception as e:
                        print(f'{e} Error Raised While Opening File')
            
                
                
            

    if collect_df:
        master_df = pd.DataFrame(columns=collect_df[0].columns)
        for df in collect_df:
            master_df = pd.concat([master_df, df], axis=0)
        print("All the Data Is Appended In Master DF: ")
        return master_df
    else:
        print('No Valid Files were Provided. ')

#### Here We Are Now taking input of our mapping column from Users

In [3]:
#### Here We Are Now taking input of our mapping column from Users
def creating_mapping(df,mapped_columns):
    global important_columns_list
    important_columns=[]
    mapping_dict={}
    print(f'We Will Be Creating Mapping For {mapped_columns.keys()}, Enter None If We Dont Have A Mapping Column For A Specific Column')
    for column in mapped_columns:
        #   ----------------------------if column name is description type--------------------------------
        if column=='Description':
            while True:
                while True:
                    invalid=[]
                    value=input(f'Enter the mapping column for {column}:  ')
                        
                    values=[col.strip() for col in value.split(',')]
                    for value in values:
                        if value not in df.columns:
                            invalid.append(value)
                    if len(invalid)!=0:
                        print(f'{invalid} not present in master file')
                        print(f'Choose From {df.columns}')
                    else:
                        break
                mapping_dict[column]=values
                important_columns.append(values)
                break

    #   --------------------------------if column name is not description type--------------------------------      
        else:
            while True:
                value=input(f'Enter the mapping column for {column},(None if no mapping column): ')
                if not value.strip():
                    print('Column Name Cannot Be Empty')
                    print(f'Choose from {df.columns}')
                else:
                    if value.lower().strip()=='none':
                        mapping_dict[column]=None
                        break
                    
                    elif value not in df.columns:
                        print(f'{value} not present in Master file')
                        print(f'Choose From {df.columns}')

                    else:
                        mapping_dict[column]=value
                        important_columns.append(value)
                        break

                    
    for column in important_columns:
        if isinstance(column,list):
            for col in column:
                important_columns_list.append(col)
        else:
            important_columns_list.append(column)
        
        
    return mapping_dict,important_columns_list

#### Beginning the analysis of the columns

In [4]:
#### Beginning the analysis of the columns
def begin_analysis(mapping_dict,cols_we_want,df,total_transactions):
    global date_format
    global date_range
    upper_r=date_range['Upper']
    lower_r=date_range['Lower']
    result={}
    for key in mapping_dict:
        # analysis of the description columns-------------->
        if key=='Description':
            desc_cols=mapping_dict[key]
            for column in  desc_cols:
                nas=pd.to_numeric(df[column],errors='coerce').notnull().sum()
                percentage=((total_transactions-nas)/total_transactions)*100
                result[column]={'Percentage Population':percentage,'Comment':None,
                               'Column Type':'Important'}

        #: analysis of Date column--------------->
        elif key=='Date':
            date_col=mapping_dict['Date']
            df['Document Date'] = pd.to_datetime(df['Document Date'],format=f'{date_format}',dayfirst=True)
            invalid_dates_count=df[~df['Document Date'].between(f'01/01/{lower_r}',f'13/09/{upper_r}')].shape[0]
            nas=df[date_col].isna().sum()
            percentage=((total_transactions-nas)/total_transactions)*100
            result[key]={'Percentage Population':percentage,'Comment':f'Total Invalid Document Dates - {invalid_dates_count}',
                               'Column Type':'Important'}
            
            
            
        elif key=='Spend':
            spend_col=mapping_dict['Spend']
            nas=df[spend_col].isna().sum()
            negative_spend=df[df['Spend']<0]['Spend'].sum()
            positive_spend=df[df['Spend']>0]['Spend'].sum()
            result[key]={'Percentage Population':percentage,'Comment':f'Negative Spend = {negative_spend} Positive Spend = {positive_spend}',
                               'Column Type':'Important'}
            
        else:
            value=mapping_dict[key]
            if value is None:
                result[key]={'Percentage Population':None,'Comment':None,
                               'Column Type':'Important'}
            else:
                value=mapping_dict[key]
                nas=df[value].isna().sum()
                percentage=((total_transactions-nas)/total_transactions)*100
                result[key]={'Percentage Population':percentage,'Comment':None,
                               'Column Type':'Important'}


#: Analysis of all the Good To have columns
    for column in df.columns:
        if column not in cols_we_want:
            nas=df[column].isna().sum()
            percentage=((total_transactions-nas)/total_transactions)*100
            result[column]={'Percentage Population':percentage,'Comment':None,
                               'Column Type':'Good To Have'}


    return result

In [5]:
def cleaning_df(df,mapping_dict):
    invalids=["#N/A",'N/A','N/A','NA','NULL','NONE','NOT ASSIGNED','NOT AVAILABLE'," "]
    special_char_pattern = re.compile(r'[^A-Za-z0-9\s]')
    description_cols=mapping_dict['Description']
    for col in description_cols:
        df[col]=df[col].replace(invalids,None)
        df[col]=df[col].apply(lambda x: re.sub(special_char_pattern,'',x) if isinstance(x,str) else x)
    print("Description Columns Cleansed.........")
    return df

In [6]:
def main():
    global date_range
    global important_columns_list
    important_columns_list=[]
    date_range={}
    date_range['Lower']='2015'
    date_range['Upper']='2024'
    global date_format
    while True:
        date_format=input("Provide the format of the date column(MM/DD/YYYY,YYYY/MM/DD,DD/YYYY/MM,MM/YYYY/DD,DD/MM/YYYY): ")
        if not date_format.strip():
            print("Format Cannot Be Null, Please Choose From Provided Formats.")
        else:
            break

    analyzing_cols =['Date',
                  "Spend",
                  "Currency",
                  "Invoice Number",
                  "Invoice Line Number",
                  "Supplier Name",
                 'Description']
    mapped_columns={}
    for col in analyzing_cols:
        mapped_columns[col]=None

    # ---------------------------------taking the input of the path-----------------------------------------
    print('taking the input of the files:--->')
    while True:
        try:
            no_files = int(input('Enter the Number of Master File We Have: '))

            if isinstance(no_files,int):
                    break

        except Exception as e:
            print(f'{e} error raised in the input, Integer Value is Expected')



    if no_files == 1:
        while True:
            file_path = input('Enter the file path for master file: ')
            if not file_path.strip():
                print('File Path Cannot Be Empty')
            else:
                
                try:
                    if file_path.strip().endswith('.csv'):
                        df = pl.read_csv(file_path)
                        df=df.to_pandas()
                        break
                        
                    else:
                        df = pl.read_excel(file_path)
                        df=df.to_pandas()
                        break
                        
                except Exception as e:
                    print(f'{e} Error While Opening File File: ')
    else:
        df=append_files(no_files)

    print('--------------------------Initiating the Mapping Process----------------------------------------')               

    mapping_dict,columns_we_want=creating_mapping(df,mapped_columns)
    
    df=cleaning_df(df,mapping_dict)

    print('-------------------------Initiating the analysis-----------------------------------------------')
    print(f'Total Number of Transactions are:  {df.shape[0]}')
    result=begin_analysis(mapping_dict,columns_we_want,df,df.shape[0])
    df2=pd.DataFrame.from_records(result)
    df2=df2.transpose()
    df2.reset_index(inplace=True)
    print(df2.head(20))

    print('-----------------------Analysis End Below Are The Results-----------------------------------')
    while True:
        file_name=input('Enter a file name to save the results in: ->')
        if not file_name.strip():
            print('File Name Cannot Be Empty')
        else:
            folder='ResultsFolder'
            os.makedirs(folder,exist_ok=True)
            file_path=os.path.join(folder,f'{file_name}.csv')
            df2.to_csv(file_path,index=False)
            print('Results has been saved to the folder...........')
            break
    print("Saving the Concated Data in another Files.....")
    
    if no_files>1:
        while True:
            file_name=input(f'As We had {no_files} Master Files, So provide a file name to save the concated data: ->')
            if not file_name.strip():
                print('File Name Cannot Be Empty')
            else:
                folder='Appended File Folder'
                os.makedirs(folder,exist_ok=True)
                file_path=os.path.join(folder,f'{file_name}.csv')
                df2=pl.from_pandas(df)
                df2.to_csv(file_path,index=False)
                print('Results has been saved to the folder...........')
                break
        

In [11]:
file_path = r"C:\Users\AnkitS-Simfoni\Users\ankit-Simfoni\Automation Tasks\Task 1 - By Ashish - Finalized\Sunsource Mapping Raw data\Sunsource Invoice_Concur August consolidation.xlsx"

In [13]:
df = pl.read_excel(file_path)
df=df.to_pandas()

,SrNo,ActualSrNo,Document Unit price,Document Quantity,Amount in Document Currency,FX Rate,Spend,Cost Center Code
count,22774.000000,22774.000000,5381.000000,5910.000000,22774.000000,22774.000000,22774.000000,20923.000000
mean,823422.500000,7011.183147,925.195515,6.882937,314.189922,0.989914,311.452189,6505.684462
std,6574.431851,4906.135340,6703.832636,71.978792,3365.536550,0.051235,3365.447073,20111.113131
min,812036.000000,1.000000,-2023.190000,-1.000000,-9270.000000,0.720240,-9270.000000,0.000000
25%,817729.250000,2847.250000,44.230000,1.000000,21.000000,1.000000,20.342500,25.000000
50%,823422.500000,5694.000000,147.520000,1.000000,50.000000,1.000000,50.000000,30.000000
75%,829115.750000,11170.750000,474.570000,1.000000,125.000000,1.000000,122.142500,101.000000
max,834809.000000,16864.000000,210450.240000,3000.000000,210450.240000,1.000000,210450.240000,90122.000000


In [ ]:
df.head()

In [8]:
df.descrine()

NameError: name 'df' is not defined

In [7]:
if __name__=="__main__":
    main()
    

Provide the format of the date column(MM/DD/YYYY,YYYY/MM/DD,DD/YYYY/MM,MM/YYYY/DD,DD/MM/YYYY):  DD/MM/YYYY


taking the input of the files:--->


Enter the Number of Master File We Have:  1
Enter the file path for master file:  C:\Users\AnkitS-Simfoni\Users\ankit-Simfoni\Automation Tasks\Task 1 - By Ashish - Finalized\Sunsource Mapping Raw data\Sunsource Invoice_Concur August consolidation.xlsx


--------------------------Initiating the Mapping Process----------------------------------------
We Will Be Creating Mapping For dict_keys(['Date', 'Spend', 'Currency', 'Invoice Number', 'Invoice Line Number', 'Supplier Name', 'Description']), Enter None If We Dont Have A Mapping Column For A Specific Column


Enter the mapping column for Date,(None if no mapping column):  Document Date
Enter the mapping column for Spend,(None if no mapping column):  Spend
Enter the mapping column for Currency,(None if no mapping column):  Document Currency
Enter the mapping column for Invoice Number,(None if no mapping column):  Document Number
Enter the mapping column for Invoice Line Number,(None if no mapping column):  Document Line Number
Enter the mapping column for Supplier Name,(None if no mapping column):  Supplier Name
Enter the mapping column for Description:   Supplier Name (Normalized)


Description Columns Cleansed.........
-------------------------Initiating the analysis-----------------------------------------------
Total Number of Transactions are:  22774
                          index Percentage Population Comment   Column Type
0                    ActualSrNo                 100.0    None  Good To Have
1           Addressability Flag                   0.0    None  Good To Have
2   Amount in Document Currency                 100.0    None  Good To Have
3                      Batch ID                   0.0    None  Good To Have
4                    Buyer Name             25.950645    None  Good To Have
5              Category Level 0                   0.0    None  Good To Have
6              Category Level 1                   0.0    None  Good To Have
7              Category Level 2                   0.0    None  Good To Have
8              Category Level 3                   0.0    None  Good To Have
9              Category Level 4                   0.0    None  Go

Enter a file name to save the results in: -> Sunsource dad Results


Results has been saved to the folder...........
Saving the Concated Data in another Files.....
